<a href="https://colab.research.google.com/github/ismaelithalo/Estudos.py/blob/master/Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q-Learning com Gym

Adaptado do [tutorial de S. Kansal e B. Martin](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)

Abrindo e mostrando um ambiente pronto do gym que simula o problema do Taxi (versão 2):


In [ ]:
import gym

env = gym.make("Taxi-v2").env

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Agente:

* Retângulo amarelo ou verde (fica verde quando está levando um passageiro)

### Ambiente:

- 16 possíveis posições possíveis
- 4 posições fixas para pegar (saída) ou largar (chegada) o passageiro (R,G,Y,B)
- Cor azul indica local de saída do passageiro e cor rosa o local de chegada
- Alguns muros indicados pelo símbolo "|"

### Estados:

500 possíveis combinações que o ambiente pode assumir

### Ações

* 0 = sul (mover taxi para baixo)
* 1 = norte (mover taxi para cima)
* 2 = leste (mover taxi para direita)
* 3 = oeste (mover taxi para esquerda)
* 4 = pegar o passageiro
* 5 = largar o passageiro

In [ ]:
print("Total de Ações {}".format(env.action_space))
print("Total de Estados {}".format(env.observation_space))

Total de Ações Discrete(6)
Total de Estados Discrete(500)


### Alterando o estado

Use a função env.encode(taxi_linha,taxi_coluna,passageiro_saida,passageiro_chegada), o local de saída é um valor de 0 a 3 indicado cada uma das 4 possíveis posições de saída e chegada: R (0), G (1), Y (2) ou B (3).

In [ ]:
state = env.encode(3, 1, 2, 1)
print("Número do estado:", state)

env.s = state
env.render()

env.s = 369
print("Número do estado:", env.s)
env.render()

Número do estado: 329
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Número do estado: 369
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Função de reforço

Valor de reforço para cada combinação de estado X ação (ou seja, uma tabela com 500 * 6 = 3000 posições no caso deste problema)

[(sempre_um, próximo_estado, valor_do_reforço, atingiu_o_objetivo)]

Possíveis punições:
* -1 : Cada movimento feito pelo carro ou tentativa de bater no muro
* -10 : Pegar ou largar o passageiro no lugar errado

Possíveis recompensas:
* +20 : Deixar o passageiro no lugar certo


In [ ]:
env.P[329]

{0: [(1.0, 429, -1, False)],
 1: [(1.0, 229, -1, False)],
 2: [(1.0, 349, -1, False)],
 3: [(1.0, 329, -1, False)],
 4: [(1.0, 329, -10, False)],
 5: [(1.0, 329, -10, False)]}

### Resolvendo usando Ações Aleatórias

In [ ]:
env.s = 329  # começa no estado do exemplo acima

epochs = 0   # total de ações realizadas
penalties = 0   # quantidade de punições recebidas por pegar ou largar no lugar errado

frames = [] # usado para fazer uma animação

done = False

while not done:
    action = env.action_space.sample()  # escolhe aleatoriamente uma ação
    state, reward, done, info = env.step(action)  # aplica a ação e pega o resultado

    if reward == -10:  # conta uma punição
        penalties += 1

    # Quarda a sequência para poder fazer a animação depois
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))



Total de ações executadas: 300
Total de penalizações recebidas: 72


### Mostrando a animação dos movimentos realizados

In [ ]:

from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 300
State: 85
Action: 5
Reward: 20


### Aprendendo a resolver usando Q-Learning

Tenho uma vídeo explicando o Q-Learning aqui: https://youtu.be/zQUFxZsZODY

In [ ]:
import numpy as np

# Inicialização com a tabela de valores Q
q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random
from IPython.display import clear_output

# Hiperparâmetros
alpha = 0.1   # taxa de aprendizagem
gamma = 0.6   # fator de desconto
epsilon = 0.1  # chance de escolha aleatória

# Total geral de ações executadas e penalidades recebidas durante a aprendizagem
epochs, penalties = 0,0

for i in range(1, 100001): # Vai rodar 100000 diferentes versões do problema
    state = env.reset()  # Inicialização aleatoria do ambient
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Escolhe ação aleatoriamente
        else:
            action = np.argmax(q_table[state]) # Escolhe ação com base no que já aprendeu

        next_state, reward, done, info = env.step(action) # Aplica a ação

        old_value = q_table[state, action]  # Valor da ação escolhida no estado atual
        next_max = np.max(q_table[next_state]) # Melhor valor no próximo estado

        # Atualize o valor Q usando a fórmula principal do Q-Learning
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:  # Contabiliza as punições por pegar ou deixar no lugar errado
            penalties += 1

        state = next_state # Muda de estado
        epochs += 1


print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))

Total de ações executadas: 1523452
Total de penalizações recebidas: 46360


### Mostrando a tabela Q para o estado 329

In [ ]:
env.s = 329
env.render()

q_table[329]

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


array([ -2.48874372,  -2.47061344,  -2.48848242,  -2.48086628,
       -10.9198647 , -11.11583968])

### Resolvendo o problema com o aprendizado adquirido


In [ ]:
state = 329
epochs, penalties = 0, 0

done = False

while not done:
     action = np.argmax(q_table[state])
     state, reward, done, info = env.step(action)

     if reward == -10:
        penalties += 1

     epochs += 1

print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))

Total de ações executadas: 14
Total de penalizações recebidas: 0


### Calculando o desempenho médio para várias versões do problema


In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Resultados depois de {episodes} simulações:")
print(f"Média de ações por simulação: {total_epochs / episodes}")
print(f"Média de penalidades: {total_penalties / episodes}")

Resultados depois de 100 simulações:
Média de ações por simulação: 12.37
Média de penalidades: 0.0
